The following code will take the CLI commands produced in `01-JJA-L2V-Configuration-Files` notebook

+ You need to install aws cli    
http://docs.aws.amazon.com/cli/latest/userguide/installing.html

+ You need to run** `aws config` **

Let's import the functions defined before for loading parameters

In [1]:
from load_config import params_to_cli

In [2]:
llr, emb, pred = params_to_cli("CONFIGS/ex1-ml-1m-config.yml", "CONFIGS/test-params.yml")

In [3]:
llr

'spark-submit --deploy-mode cluster --class llr.LLR s3://sandbox-l2v/JARs/llr-assembly-1.2.jar --master yarn --options default --useroritem user --threshold 0.5 --interactionsFile s3://sandbox-l2v/datasets/ml-1m/split/split-cleaned-formatted-4and5/ml1m-train-clean4and5 --outputFile s3://sandbox-l2v/datasets/ml-1m/llr_output/llr12-081017-du05 --separator "," --maxInteractionsPerUserOrItem 500 --seed 12345'

This function will format the AWS CLI commands so we can pass them to the cluster using `boto3`

In [4]:
def create_steps(llr=None, emb=None, pred=None, name=''):
    if llr != None:
        Steps=[

        {
            'Name': name + '-LLR',
            'ActionOnFailure': 'CONTINUE',
            'HadoopJarStep': {
                'Jar': 'command-runner.jar',
                'Args': (llr).split(),
            }
        },
        {
            'Name': name + '-EMB',
            'ActionOnFailure': 'CONTINUE',
            'HadoopJarStep': {
                'Jar': 'command-runner.jar',
                'Args': (emb).split(),
            }
        },
        {
            'Name': name + '-PRED',
            'ActionOnFailure': 'CONTINUE',
            'HadoopJarStep': {
                'Jar': 'command-runner.jar',
                'Args': (pred).split(),
            }
        }
    ]
    else:
        Steps=[
        {
            'Name': name + '-EMB',
            'ActionOnFailure': 'CONTINUE',
            'HadoopJarStep': {
                'Jar': 'command-runner.jar',
                'Args': (emb).split(),
            }
        },
        {
            'Name': name + '-PRED',
            'ActionOnFailure': 'CONTINUE',
            'HadoopJarStep': {
                'Jar': 'command-runner.jar',
                'Args': (pred).split(),
            }
        }
    ]
                     
            
    return Steps
    

To load the commands into EMR

Here we create steps based on the three steps in the pipeline

In [5]:
ex2 = create_steps(llr=llr, emb=emb, pred=pred, name='EXP2')
# ex3 = create_steps(emb=emb347, pred=pred347, name='EXP3')
# ex4 = create_steps(emb=emb348, pred=pred348, name='EXP4')
# ex5 = create_steps(emb=emb349, pred=pred349, name='EXP5')

If we are adding multiple runs of the pipeline

In [6]:
# steps = ex2 + ex3 + ex4 + ex5
steps = ex2

To run the steps into EMR using boto3

In [7]:
import boto3

In [8]:
client = boto3.client('emr')

In [9]:
cluster_id = 'j-3OPHOXTNBHNLC'

In [10]:
response = client.add_job_flow_steps(
    JobFlowId = cluster_id,
    Steps= steps
)

In [11]:
response

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '66',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 10 Aug 2017 21:35:19 GMT',
   'x-amzn-requestid': 'ce51dfcc-7e13-11e7-8f91-cb7d288c1524'},
  'HTTPStatusCode': 200,
  'RequestId': 'ce51dfcc-7e13-11e7-8f91-cb7d288c1524',
  'RetryAttempts': 0},
 'StepIds': ['s-2QE3QVH7JT97U', 's-1EJYM23D8X4QL', 's-Y0J80Q60ZGFF']}